In [1]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import struct
from scipy.io import wavfile as wav
import os
from datetime import datetime 

from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint 
#### Dependencies ####

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccsscaled

# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'UrbanSound8K/audio/'

metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')

features = []

for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class"]
    data = extract_features(file_name)
    
    features.append([data, class_label])
    
# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

num_epochs = 100
num_batch_size = 32

start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)

duration = datetime.now() - start

print("Training completed in time: ", duration)

score = model.evaluate(x_train, y_train, verbose=0)
print('train accuracy: {}'.format(score))
# experiment.log_metric("train_acc", score)

score = model.evaluate(x_test, y_test, verbose=0)
print('test accuracy: {}'.format(score))
# experiment.log_metric("val_acc", score)


Train on 6985 samples, validate on 1747 samples
Epoch 1/100
6985/6985 [==============================] - 1s 87us/step - loss: 8.4859 - accuracy: 0.1983 - val_loss: 2.1263 - val_accuracy: 0.2376
Epoch 2/100
6985/6985 [==============================] - 0s 57us/step - loss: 2.2244 - accuracy: 0.2634 - val_loss: 2.0446 - val_accuracy: 0.2811
Epoch 3/100
6985/6985 [==============================] - 0s 56us/step - loss: 2.0277 - accuracy: 0.3057 - val_loss: 1.8436 - val_accuracy: 0.3692
Epoch 4/100
6985/6985 [==============================] - 0s 59us/step - loss: 1.8536 - accuracy: 0.3556 - val_loss: 1.6354 - val_accuracy: 0.4585
Epoch 5/100
6985/6985 [==============================] - 0s 56us/step - loss: 1.7056 - accuracy: 0.4010 - val_loss: 1.4708 - val_accuracy: 0.5426
Epoch 6/100
6985/6985 [==============================] - 0s 58us/step - loss: 1.6089 - accuracy: 0.4448 - val_loss: 1.3543 - val_accuracy: 0.5615
Epoch 7/100
6985/6985 [==============================] - 0s 56us/step - los

6985/6985 [==============================] - 0s 57us/step - loss: 0.6219 - accuracy: 0.7931 - val_loss: 0.5027 - val_accuracy: 0.8626
Epoch 56/100
6985/6985 [==============================] - 0s 56us/step - loss: 0.6226 - accuracy: 0.7940 - val_loss: 0.4927 - val_accuracy: 0.8609
Epoch 57/100
6985/6985 [==============================] - 0s 55us/step - loss: 0.6288 - accuracy: 0.7877 - val_loss: 0.5026 - val_accuracy: 0.8540
Epoch 58/100
6985/6985 [==============================] - 0s 57us/step - loss: 0.6352 - accuracy: 0.7873 - val_loss: 0.5092 - val_accuracy: 0.8477
Epoch 59/100
6985/6985 [==============================] - 0s 56us/step - loss: 0.6263 - accuracy: 0.7943 - val_loss: 0.4823 - val_accuracy: 0.8609
Epoch 60/100
6985/6985 [==============================] - 0s 56us/step - loss: 0.6102 - accuracy: 0.7917 - val_loss: 0.5049 - val_accuracy: 0.8546
Epoch 61/100
6985/6985 [==============================] - 0s 57us/step - loss: 0.6139 - accuracy: 0.7900 - val_loss: 0.4688 - val_a